To train a neural network, we need a database of identically formatted fire tensors.

This notebook scans a fire perimeter for interesting convolutions, 256x256 30m squares where there is a mix of burning and non-burning areas, the fire has not overrun the boundaries, which would cause training errors. The results are then stored in a MongoDB for easy access.

In [111]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymongo as pm
import pyprind
import collections
import matplotlib as mpl
import pickle
import datetime as dt
import geopandas as gpd

cmap = mpl.colors.ListedColormap(['black', 'red', 'yellow', 'blue', 'green'])

%matplotlib inline

FBFM40 is the Scott and Burgan Fire Behavior Fuel Model, which describes flammable materials on 40 landscape types. 

In [71]:
FBFM40 = pd.read_csv('FBFM40convert.csv', header=0, index_col=0)

fbfm401hr = dict()
fbfm4010hr = dict()
fbfm40100hr = dict()
fbfm40live = dict()
fbfm40woody = dict()
fbfm40dead = dict ()
for row in FBFM40.itertuples():
    fbfm401hr[row[0]]   = row[2]
    fbfm4010hr[row[0]]  = row[3]
    fbfm40100hr[row[0]] = row[4]
    fbfm40live[row[0]]  = row[5]
    fbfm40woody[row[0]] = row[6]
    fbfm40dead[row[0]]  = row[7]/100

fbfm = np.zeros([205,6])

for item in fbfm401hr:
    fbfm[item,0] = fbfm401hr[item]
    fbfm[item,1] = fbfm4010hr[item]
    fbfm[item,2] = fbfm40100hr[item]
    fbfm[item,3] = fbfm40live[item]
    fbfm[item,4] = fbfm40woody[item]
    fbfm[item,5] = fbfm40dead[item]

In [72]:
def convolute(raster, coords):
    """
    Takes in a raster and a coordinate list of [left, right, top, bottom]
    Returns convolution"""
    return raster[coords[2]:coords[3],coords[0]:coords[1]]

def convolute3d(raster, coords):
    """
    Takes in a raster and a coordinate list of [left, right, top, bottom, t0, t1]
    Returns convolution"""
    return raster[coords[4]:coords[5],coords[2]:coords[3],coords[0]:coords[1]]

def coord_gen(shape, conv_size, stride):
    """
    Takes in a np.shape, and desired convolution size and stride.
    Returns a valid list of coordinates for that shape
    """
    coords = []
    xs = np.arange(0, shape[1]-conv_size, stride)
    ys = np.arange(0, shape[0]-conv_size, stride)
    for y in ys:
        for x in xs:
            new_coord = [x, x+conv_size, y, y+conv_size]
            coords.append(new_coord)
    return(coords)

def perimeter_finder(ar, br, conv_size, stride):
    """
    ar and br are two fire perimeters from the same fire separated by a reasonable interval
    conv_size and stride and the convolution size and stride
    """
    
    interesting_coords = []

    for coord in coord_gen(ar.shape, conv_size, stride):
        ac = convolute(ar, coord)
        bc = convolute(br, coord)
        max_area = ac.shape[0]*ac.shape[1]
        if 0 < np.sum(ac) and np.sum(ac) < max_area:
            interesting_coords.append(coord)
    return interesting_coords

def azimuthx(a):
    return np.cos(np.radians(a))

def azimuthy(a):
    return np.sin(np.radians(a))

def bytes_to_raster(buf, shape):
    br = np.frombuffer(buf, dtype=np.float)
    r = br.reshape(shape)
    return r

def db_update(ic):
    c_perim1 = convolute(perim1['raster'], ic)
    c_perim0 = convolute(perim0['raster'], ic)
    c_dem = convolute(topo['us_dem'], ic)
    c_slp = convolute(topo['us_slp'], ic)  
    c_asp = convolute(topo['us_asp'], ic)  
    c_fbfm40 = convolute(topo['us_130fbfm40'], ic)  

    c=np.zeros([12, 256, 256])

    c[0,:,:] = c_perim1
    c[1,:,:] = c_perim0
    c[2,:,:] = c_dem-np.mean(c_dem)
    c[3,:,:] = azimuthx(c_asp)
    c[4,:,:] = azimuthy(c_asp)
    c[5,:,:] = c_slp
    for i in range(256):
        for j in range(256):
            c[6:12,i,j]= fbfm[c_fbfm40[i,j]]


    topo_shape = perim1['raster'].shape

    enddate = dt.datetime.strptime(atmo['enddate'], '%Y-%m-%d')
    startdate = dt.datetime.strptime(atmo['startdate'], '%Y-%m-%d')

    zero = startdate + dt.timedelta(hours=-1000)
    now = dt.datetime.strptime(perim0['date'], '%Y-%m-%d')
    predict = dt.datetime.strptime(perim1['date'], '%Y-%m-%d')

    t1 = enddate-predict

    t1_index = t1.days*24

    t0 = enddate-now

    t0_index = t0.days*24

    atmo_shape = atmo['windspeed'].shape[1:]
    conversion_factor = np.mean(topo_shape[0]/atmo_shape[0]+topo_shape[1]/atmo_shape[1])
    atmo_coords = np.round(ic/conversion_factor, 0)
    atmo_coords = [int(x) for x in atmo_coords]
    atmo_coords = [-t0_index, -t1_index] + atmo_coords
    ac =atmo_coords
    if ac[1] == 0:
        ac[1] = -1
        ac[0] = ac[0] -1

    c_winddir   = atmo['winddirection'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
    c_windspeed = atmo['windspeed'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
    c_precip    = atmo['precipitation'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
    c_humid     = atmo['humidity'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
    c_temp      = atmo['temperature'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]

    a = np.zeros([6, t0_index-t1_index, c_winddir.shape[1], c_winddir.shape[2]])

    a[0, :, :, :] = azimuthx(c_winddir)
    a[1, :, :, :] = azimuthy(c_winddir)
    a[2, :, :, :] = c_windspeed
    a[3  :, :, :] = c_precip
    a[4, :, :, :] = c_humid
    a[5, :, :, :] = c_temp -273

    date = now.strftime('%Y-%d-%m')

    data = {
    'fire_id':fire_id,
    'date': date,
    't_shape' : c.shape,
    'a_shape' : a.shape,
    'topo' : c.tobytes(),
    'atmo' : a.tobytes()
    }
    return data

In [73]:
atmo_files=os.listdir('wind')
topo_files=os.listdir('topo')
perim_files = os.listdir('perims/clean')

In [74]:
#these two fires cause problems, due to their enormous size.

fire_ids =  [x.split(' ')[0] for x in atmo_files]
print(len(fire_ids))
fire_ids.remove('ID-SCF-G4A0')
fire_ids.remove('WA-OWF-G70S')
print(len(fire_ids))

139
137


In [19]:
# Counting the number of interesting convolutions. Can be skipped.

# num_conv = 0

# progbar = pyprind.ProgBar(len(fire_ids), width=80, update_interval=5)

# for fire_id in fire_ids:

#     layers = ['us_slp', 'us_dem', 'us_asp', 'us_130fbfm40']

#     for layer in layers:
#         topo[layer] = topo[layer].T

#     perims = [x for x in perim_files if fire_id in x]
#     perims.sort()

#     for s in range(len(perims)-1):
#         op = open('perims/clean/'+perims[s+1], 'rb')
#         perim0 = pickle.load(op)
#         op.close()

#         op = open('perims/clean/'+perims[s], 'rb')
#         perim1 = pickle.load(op)
#         op.close()

#         perim0['raster'] = perim0['raster'].astype(int)
#         perim1['raster'] = perim1['raster'].astype(int)

#         ics = perimeter_finder(perim0['raster'], perim1['raster'], 256, 32)
#         num_conv +=len(ics)
#     progbar.update()

0% [#############################################################################   ] 100% | ETA: 00:00:09
Total time elapsed: 00:04:21


In [81]:
#if you want to run this quickly
num_conv = 642893

num_conv

642893

In [91]:
client = pm.MongoClient()
local = client.firemind

firemind.drop_collection('convolutions')
firemind.create_collection('convolutions')

convolutions = firemind.get_collection('convolutions')

In [93]:
errors = []

progbar = pyprind.ProgBar(num_conv, width=80, update_interval=5)

for fire_id in [fire_ids[0]]:

    op = open('wind/'+fire_id+' wind.pkl', 'rb')
    atmo = pickle.load(op)
    op.close()

    op = open('topo/'+fire_id+' topo.pkl', 'rb')
    topo = pickle.load(op)
    op.close()

    layers = ['us_slp', 'us_dem', 'us_asp', 'us_130fbfm40']

    for layer in layers:
        topo[layer] = topo[layer].T

    perims = [x for x in perim_files if fire_id in x]
    perims.sort()

    for s in range(len(perims)-1):
        op = open('perims/clean/'+perims[s], 'rb')
        perim0 = pickle.load(op)
        op.close()

        op = open('perims/clean/'+perims[s+1], 'rb')
        perim1 = pickle.load(op)
        op.close()

        perim0['raster'] = perim0['raster'].astype(int)
        perim1['raster'] = perim1['raster'].astype(int)
        #print(perim1['date'], perim0['date'])

        ics = perimeter_finder(perim0['raster'], perim1['raster'], 256, 32)

        for ic in ics:
            c_perim1 = convolute(perim1['raster'], ic)
            c_perim0 = convolute(perim0['raster'], ic)
            c_dem = convolute(topo['us_dem'], ic)
            c_slp = convolute(topo['us_slp'], ic)  
            c_asp = convolute(topo['us_asp'], ic)  
            c_fbfm40 = convolute(topo['us_130fbfm40'], ic)  

            c=np.zeros([12, 256, 256])

            c[0,:,:] = c_perim1
            c[1,:,:] = c_perim0
            c[2,:,:] = c_dem-np.mean(c_dem)
            c[3,:,:] = azimuthx(c_asp)
            c[4,:,:] = azimuthy(c_asp)
            c[5,:,:] = c_slp
            for i in range(256):
                for j in range(256):
                    c[6:12,i,j]= fbfm[c_fbfm40[i,j]]


            topo_shape = perim1['raster'].shape

            enddate = dt.datetime.strptime(atmo['enddate'], '%Y-%m-%d')
            startdate = dt.datetime.strptime(atmo['startdate'], '%Y-%m-%d')

            zero = startdate + dt.timedelta(hours=-1000)
            
            now = dt.datetime.strptime(perim0['date'], '%Y-%m-%d')
            predict = dt.datetime.strptime(perim1['date'], '%Y-%m-%d')

            t0 = int((predict-startdate).total_seconds()/3600+900)

            t1_index = int((now-startdate).total_seconds()/3600+1000)

            atmo_shape = atmo['windspeed'].shape[1:]
            conversion_factor = np.mean(topo_shape[0]/atmo_shape[0]+topo_shape[1]/atmo_shape[1])
            atmo_coords = np.round(ic/conversion_factor, 0)
            atmo_coords = [int(x) for x in atmo_coords]
            atmo_coords = [t0_index, t1_index] + atmo_coords
            ac =atmo_coords

            c_winddir   = atmo['winddirection'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
            c_windspeed = atmo['windspeed'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
            c_precip    = atmo['precipitation'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
            c_humid     = atmo['humidity'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
            c_temp      = atmo['temperature'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]

            a = np.zeros([6, t1_index-t0_index, c_winddir.shape[1], c_winddir.shape[2]])

            a[0, :, :, :] = azimuthx(c_winddir)
            a[1, :, :, :] = azimuthy(c_winddir)
            a[2, :, :, :] = c_windspeed
            a[3  :, :, :] = c_precip
            a[4, :, :, :] = c_humid
            a[5, :, :, :] = c_temp -273

            date = now.strftime('%Y-%d-%m')

            data = {
            'fire_id':fire_id,
            'date': date,
            't_shape' : c.shape,
            'a_shape' : a.shape,
            'topo' : c.tobytes(),
            'atmo' : a.tobytes()
            }
            convolutions.insert_one(data)
            progbar.update()

0% [                                                                                ] 100% | ETA: 22:24:36

In [95]:
convolutions.estimated_document_count()

/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


1030